#https://www.youtube.com/watch?v=SvfLORe3KHo
#https://www.youtube.com/watch?v=RhLUyJxS8Tk
import json

print('Loading function')

def lambda_handler(event, context):
	print('------------------------')
	print(event)
	#1. Iterate over each record
	try:
		for record in event['Records']:
			#2. Handle event by type
			if record['eventName'] == 'INSERT':
				handle_insert(record)
			elif record['eventName'] == 'MODIFY':
				handle_modify(record)
			elif record['eventName'] == 'REMOVE':
				handle_remove(record)
		print('------------------------')
		return "Success!"
	except Exception as e: 
		print(e)
		print('------------------------')
		return "Error"


def handle_insert(record):
	print("Handling INSERT Event")
	
	#3a. Get newImage content
	newImage = record['dynamodb']['NewImage']
	
	#3b. Parse values
	newPlayerId = newImage['playerId']['S']

	#3c. Print it
	print ('New row added with playerId=' + newPlayerId)

	print("Done handling INSERT Event")

def handle_modify(record):
	print("Handling MODIFY Event")

	#3a. Parse oldImage and score
	oldImage = record['dynamodb']['OldImage']
	oldScore = oldImage['score']['N']
	
	#3b. Parse oldImage and score
	newImage = record['dynamodb']['NewImage']
	newScore = newImage['score']['N']

	#3c. Check for change
	if oldScore != newScore:
		print('Scores changed - oldScore=' + str(oldScore) + ', newScore=' + str(newScore))

	print("Done handling MODIFY Event")

def handle_remove(record):
	print("Handling REMOVE Event")

	#3a. Parse oldImage
	oldImage = record['dynamodb']['OldImage']
	
	#3b. Parse values
	oldPlayerId = oldImage['playerId']['S']

	#3c. Print it
	print ('Row removed with playerId=' + oldPlayerId)

	print("Done handling REMOVE Event")

In [55]:
import json
import boto3
import requests
import re
from bson import json_util

In [9]:
client = boto3.client('firehose')

In [10]:
response = client.describe_delivery_stream(DeliveryStreamName='ddb-stream-to-s3', Limit = 123)

Describe = json.dumps(response, indent = 3, default=json_util.default)

In [123]:
event =  {'eventID': '277688d61b272be8574fe2d2186be3c7', 'eventName': 'INSERT', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'ap-southeast-1', 'dynamodb': {'ApproximateCreationDateTime': 1625458660.0, 'Keys': {'connection': {'S': 'thanhluong1'}}, 'NewImage': {'password': {'N': '123456'}, 'connection': {'S': 'thanhluong1'}}, 'SequenceNumber': '14300000000005329690951', 'SizeBytes': 54, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:ap-southeast-1:234810900335:table/lambdaTrigger/stream/2021-07-05T02:36:42.776'}

In [124]:
event.keys()

dict_keys(['eventID', 'eventName', 'eventVersion', 'eventSource', 'awsRegion', 'dynamodb', 'eventSourceARN'])

In [125]:
new_connection = event['dynamodb']['NewImage']['connection']['S']
new_connection

'thanhluong1'

In [126]:
new_password = int(event['dynamodb']['NewImage']['password']['N'])
new_password

123456

In [127]:
data = {'connection':'%s' %new_connection, 'password':'%d' %new_password}
print(data)
print(type(data))

{'connection': 'thanhluong1', 'password': '123456'}
<class 'dict'>


In [128]:
d = json.dumps(data)
d = d+"\n"
data = d.encode("utf-8")
print(data)

b'{"connection": "thanhluong1", "password": "123456"}\n'


In [129]:
client = boto3.client('firehose')
response = client.put_record(DeliveryStreamName='ddb-stream-to-s3', Record={'Data':data})
print(json.dumps(response, indent = 3))

{
   "RecordId": "udZUauxYQ/wyqiXo3uwDO08/pSxDSNQz7X0YVwn5bVjDRZ7W6hc4OBSEuEErqVo87kAhmSAlVbl+lKRXvuLh45tTOmKYH7fbaw6P04CIYWRoJ6C5k6eUohr26cR+TQexWj8i8ZUC8fGMWy05QgOxGRsDxXU8vgci/kwVa5NV4O89ol6gmJ9LsjjH7CpH/J81QQCwYqYsg0EdPZGjJYYPMyerJq/AZrDB",
   "Encrypted": false,
   "ResponseMetadata": {
      "RequestId": "de9701e6-2150-e10a-87cb-8bab0de141fd",
      "HTTPStatusCode": 200,
      "HTTPHeaders": {
         "x-amzn-requestid": "de9701e6-2150-e10a-87cb-8bab0de141fd",
         "x-amz-id-2": "4RXpvaDj+3FSdpJAKkXodtKLmKIz055+o0yWGPKJj8XLaoxXfLZM5HUuXu+Y6rMwGhcjYZWWnR79BzwFCzr27l7GuZhTFqUe",
         "content-type": "application/x-amz-json-1.1",
         "content-length": "257",
         "date": "Mon, 05 Jul 2021 04:18:20 GMT"
      },
      "RetryAttempts": 0
   }
}
